# FORCING

# So Idea:
- get typical (aggregated) forcing for both regimes (100m box model structure)
- instead of aggregating to one year, get forcing for 5 years per each regime


# now what to do first: 
1. get necessary raw data, 
2. then integrate over entire time-series & depth (if necessary)
3. create aggregated forcing raw csvs for model (simply include proper Date information, in addition to yday)
4. date truncation & aggregation can happen in python code of model!

In [1]:
### create one large file with all possible forcing? (depends on where data is from, don't overcomplicate)

Let's write up what I need:
1. x25.8 depth (!) 
2. T_box
3. PAR
4. N0 (mean 100-150m)
4. NH40, ....
5. Si0
6. P0


In [2]:
### important point: actually use (as much as possible) raw values as forcing, then assumptions more apparent

# NISKIN DATA

In [3]:
require(ncdf4, warn.conflicts = FALSE);
require(cowplot, warn.conflicts = FALSE);
require(tidyverse, warn.conflicts = FALSE);
require(oce, warn.conflicts = FALSE);


require(lubridate, warn.conflicts = FALSE);

Loading required package: ncdf4
Loading required package: cowplot
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************

Loading required package: tidyverse
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.3  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks

In [4]:
# open a NetCDF file
niskdat <- nc_open("../01Observations/Data/niskin.netcdf")
#print(niskdat)

In [5]:
niskdat_df <- data.frame("Num" = 1:niskdat$dim$unlimited$len)
print('start')
for(i in 1:niskdat$nvars){
    #print(i)
    niskdat_df[paste(attributes(niskdat$var)$names[i])] <- ncvar_get(niskdat, attributes(niskdat$var)$names[i])
}
print('done reading')
head(niskdat_df)

[1] "start"
[1] "done reading"


Num,Cruise_number,Cruise_ID_1,Cruise_ID_2,Leg,Day,Month,Year,Latitude,Longitude,...,Bact_Biomass_uMC,Bio_cast_no,ISO_DateTime_start_hc_local,ISO_DateTime_end_hc_local,ISO_DateTime_start_bc_local,ISO_DateTime_end_bc_local,ISO_DateTime_start_hc_UTC,ISO_DateTime_end_hc_UTC,ISO_DateTime_start_bc_UTC,ISO_DateTime_end_bc_UTC
1,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
2,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
3,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
4,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
5,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
6,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00


In [6]:
cat(names(niskdat_df))

Num Cruise_number Cruise_ID_1 Cruise_ID_2 Leg Day Month Year Latitude Longitude Hydro_cast_no Depth_target Depth_real O2_ml_L q_O2_ml_L O2_umol_kg q_O2_umol_kg NO3_UDO q_NO3_UDO PO4_UDO q_PO4_UDO SiO4_UDO q_SiO4_UDO NH4_USF q_NH4_USF NO2_USF q_NO2_USF NO3_NO2_USF q_NO3_NO3_USF PO4_USF q_PO4_USF SiO4_USF q_SiO4_USF pH q_pH Alkalinity_mol_kg q_Alkalinity_mol_kg Alkalinity_umol_kg q_Alkalinity_umol_kg TCO2 q_TCO2 fCO2 q_fCO2 pH_corrected q_pH_corrected TCO2_corrected q_TCO2_corrected fCO2_corrected q_fCO2_corrected Salinity_bottles q_Salinity_bottles Salinity_CTD q_Salinity_CTD Temperature q_Temperature Sigma_t q_Sigma_t TPP q_TPP PIP q_PIP POC_ug_kg q_POC_ug_kg PON_ug_kg q_PON_ug_kg POC_ug_L q_POC_ug_L PN_ug_L q_PN_ug_L C_N_particulate q_C_N_particulate DON q_DON DOP q_DOP DOC q_DOC TOC q_TOC PrimaryProductivity q_PrimaryProductivity Chlorophyll q_Chlorophyll Phaeopigments q_Phaeopigments Total_Prokaryotes Bact_Biomass_mgC_m3 Bact_Biomass_uMC Bio_cast_no ISO_DateTime_start_hc_local ISO_D

In [7]:
niskdat_df$Date <- paste(niskdat_df$Year,'-',niskdat_df$Month,'-',niskdat_df$Day,sep='')

In [8]:
niskdat_df$date <- as.Date(niskdat_df$Date) # as.Date(substr(niskdat_df$ISO_DateTime_end_hc_UTC, 0, 10))

In [9]:
niskdat_df$Depth_real <- as.numeric(niskdat_df$Depth_real)
niskdat_df$Depth_target <- as.numeric(niskdat_df$Depth_target)

niskdat_df$Temperature <- as.numeric(niskdat_df$Temperature)

niskdat_df$NO3_NO2_USF <- as.numeric(niskdat_df$NO3_NO2_USF)
niskdat_df$NO3_UDO <- as.numeric(niskdat_df$NO3_UDO)

niskdat_df$SiO4_USF <- as.numeric(niskdat_df$SiO4_USF)
niskdat_df$SiO4_UDO <- as.numeric(niskdat_df$SiO4_UDO)

niskdat_df$PO4_USF <- as.numeric(niskdat_df$PO4_USF)
niskdat_df$PO4_UDO <- as.numeric(niskdat_df$PO4_UDO)

niskdat_df$Sigma_t <- as.numeric(niskdat_df$Sigma_t)
niskdat_df$Salinity_bottles <- as.numeric(niskdat_df$Salinity_bottles)
niskdat_df$Salinity_CTD <- as.numeric(niskdat_df$Salinity_CTD)

In [12]:
#niskdat_df$Depth_real

In [17]:
#mean(niskdat_df[niskdat_df$Depth_real<100,]$Temperature, na.rm=T)

In [10]:
#head(niskdat_df[niskdat_df$date > as.Date('1997-05-10'),]$SiO4_USF, n=100)
#head(niskdat_df[niskdat_df$date > as.Date('1997-05-10'),]$Depth_real, n=100)

In [11]:
niskdat_df$O2_umol_kg <- as.numeric(niskdat_df$O2_umol_kg)
niskdat_df$POC_ug_kg <- as.numeric(niskdat_df$POC_ug_kg)
niskdat_df$POC_ug_L <- as.numeric(niskdat_df$POC_ug_L)
niskdat_df$PN_ug_L <- as.numeric(niskdat_df$PN_ug_L)
niskdat_df$C_N_particulate <- as.numeric(niskdat_df$C_N_particulate)

In [12]:
niskdat_df$PON_ug_kg <- as.numeric(niskdat_df$PON_ug_kg)
niskdat_df$PN_ug_L <- as.numeric(niskdat_df$PN_ug_L)

niskdat_df$C_N_particulate <- as.numeric(niskdat_df$C_N_particulate)
niskdat_df$PIP <- as.numeric(niskdat_df$PIP)
niskdat_df$TPP <- as.numeric(niskdat_df$TPP)
niskdat_df$DOC <- as.numeric(niskdat_df$DOC)
niskdat_df$DON <- as.numeric(niskdat_df$DON)
niskdat_df$DOP <- as.numeric(niskdat_df$DOP)
niskdat_df$TOC <- as.numeric(niskdat_df$TOC)

In [13]:
niskdat_df$PrimaryProductivity <- as.numeric(niskdat_df$PrimaryProductivity)
niskdat_df$Chlorophyll <- as.numeric(niskdat_df$Chlorophyll)
niskdat_df$Phaeopigments <- as.numeric(niskdat_df$Phaeopigments)
niskdat_df$Total_Prokaryotes <- as.numeric(niskdat_df$Total_Prokaryotes)
niskdat_df$Bact_Biomass_mgC_m3 <- as.numeric(niskdat_df$Bact_Biomass_mgC_m3)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [14]:
# Function to read specific verfication type from dataset
prepdataframe <- function(variable='', datasource='niskin'){
    if(datasource=='niskin'){
    DF <- niskdat_df
    DF$depth <- DF$Depth_real
    # print(names(DF))
        
    VarDF <- DF[complete.cases(DF[variable]),] %>%
      select(date, variable, depth) %>%
      gather(key='key',value = "value", -date, -depth)
    head(VarDF)
    }
    return(VarDF)
}

In [15]:
# integration functions (if necessary)

# adapt integration function from PINCKNEY
integrateJAYfunc <- function(x, y){
    N1 = length(y)
    area = 0
    
    for(j in 2:N1){
        area = area + (x[j] - x[j-1]) * (y[j] + y[j-1]) / 2
    }
    return(area)
}

intgrtJAY <- function(DFtest, reg, var){
    DF_int <- DFtest %>%
        group_by(date) %>%
        do(data.frame("value_int" = with(.,integrateJAYfunc(x=depth, y=value)), "reg" = reg , "var" = var)) %>%
        ungroup()
        
    return(DF_int)
}

In [16]:
# Interpolate
interpolateDF <- function(DF,func='oce-rr'){
    zz <- seq(0, 200, 1)
    var = DF$key[1]
    if(func=='oce-rr'){
        IntDF <- DF %>%
            group_by(date) %>%
            do(data.frame(value_int = with(.,oceApprox(depth, value, zz, "rr")), depth = zz)) 
        }
    
    return(IntDF)
}

# take MEAN of concentrations at certain depth
meanTOdepth <- function(DF_int,type='fixed',depthFROM=0,depthTO=100){
    DF_means <- DF_int %>%
        group_by(date) %>% # this groups it by the run number 
        filter(depthFROM < depth & depth < depthTO) %>% # only keep values above the depth passed to function
        summarize('mean'=mean(value_int, na.rm=T))

    return(DF_means)}

In [17]:
extrctintrplt <- function(x, below=TRUE){
    DF <- prepdataframe(x)
    
    #head(DF)
    ##############################
    DF_int <- interpolateDF(DF)
    #head(DF_int)
    ##############################
    DF_mean <- meanTOdepth(DF_int, type='fixed')
    names(DF_mean)[2] <- paste(x,'_Box', sep='')
    #head(DF_mean)
    if(below==TRUE){
        ##############################
        DF_atDEPTH <- meanTOdepth(DF_int, type='fixed', depthFROM = 100, depthTO = 150)
        names(DF_atDEPTH)[2] <- paste(x,'_AtDepth', sep='')
        #head(DF_atDEPTH)
        ##############################
        DFX <- merge(DF_mean,DF_atDEPTH, all=T)
    }else{
        DFX <- DF_mean
    }

    return(DFX)
}

In [18]:
Temperature <- extrctintrplt('Temperature')
#head(Temp)
NO3_NO2_USF <- extrctintrplt('NO3_NO2_USF')
#head(NO3_NO2_USF)
NO3_UDO <- extrctintrplt('NO3_UDO')
#head(NO3_UDO)
SiO4_USF <- extrctintrplt('SiO4_USF')
#head(SiO4_USF)
SiO4_UDO <- extrctintrplt('SiO4_UDO')
#head(SiO4_UDO)
PO4_USF <- extrctintrplt('PO4_USF')
#head(PO4_USF)
PO4_UDO <- extrctintrplt('PO4_UDO')
#head(PO4_UDO)
Sigma_t <- extrctintrplt('Sigma_t')
#head(Sigma_t)
Salinity_bottles <- extrctintrplt('Salinity_bottles')
#head(Salinity_bottles)
Salinity_CTD <- extrctintrplt('Salinity_CTD')
#head(Salinity_CTD)

In [19]:
O2_umol_kg <- extrctintrplt('O2_umol_kg')
#head(O2_umol_kg)
POC_ug_kg <- extrctintrplt('POC_ug_kg')
#head(POC_ug_kg)
POC_ug_L <- extrctintrplt('POC_ug_L')
#head(POC_ug_L)
PN_ug_L <- extrctintrplt('PN_ug_L')
#head(PN_ug_L)
C_N_particulate <- extrctintrplt('C_N_particulate')
#head(C_N_particulate)
PON_ug_kg <- extrctintrplt('PON_ug_kg')
#head(PON_ug_kg)
PIP <- extrctintrplt('PIP')
#head(PIP)
DOC <- extrctintrplt('DOC')
#head(DOC)
DON <- extrctintrplt('DON')
#head(DON)
DOP <- extrctintrplt('DOP')
#head(DOP)
TOC <- extrctintrplt('TOC')
#head(TOC)

In [20]:
PrimaryProductivity <- extrctintrplt('PrimaryProductivity')
#head(PrimaryProductivity)
Chlorophyll <- extrctintrplt('Chlorophyll')
#head(Chlorophyll)
Phaeopigments <- extrctintrplt('Phaeopigments')
#head(DON)
Total_Prokaryotes <- extrctintrplt('Total_Prokaryotes')
#head(DOP)
Bact_Biomass_mgC_m3 <- extrctintrplt('Bact_Biomass_mgC_m3')
#head(TOC)

In [21]:
BoxVSAtDepth <- list(Temperature, NO3_NO2_USF, NO3_UDO , SiO4_USF, SiO4_UDO, PO4_USF, PO4_UDO, Sigma_t, Salinity_bottles, Salinity_CTD,
    O2_umol_kg, POC_ug_kg, POC_ug_L, PN_ug_L, C_N_particulate, PON_ug_kg, PIP, DOC, DON, DOP, TOC,
    PrimaryProductivity, Chlorophyll, Phaeopigments, Total_Prokaryotes, Bact_Biomass_mgC_m3) %>% reduce(left_join, by = "date")

In [23]:
BoxVSAtDepth$yday <- yday(BoxVSAtDepth$date)
BoxVSAtDepth$month <- month(BoxVSAtDepth$date)

In [24]:
str(BoxVSAtDepth)

'data.frame':	230 obs. of  55 variables:
 $ date                       : Date, format: "1995-11-08" "1995-12-13" ...
 $ Temperature_Box            : num  25.3 23.7 22.6 22.3 21.2 ...
 $ Temperature_AtDepth        : num  21.6 20.6 20.3 20.1 19.5 ...
 $ NO3_NO2_USF_Box            : num  NA NA NA NA NA NA NA NA NA NA ...
 $ NO3_NO2_USF_AtDepth        : num  NA NA NA NA NA NA NA NA NA NA ...
 $ NO3_UDO_Box                : num  0.86 3.07 7.48 6.79 6.5 ...
 $ NO3_UDO_AtDepth            : num  6.59 7.36 16.09 13.86 10.46 ...
 $ SiO4_USF_Box               : num  NA NA NA NA NA NA NA NA NA NA ...
 $ SiO4_USF_AtDepth           : num  NA NA NA NA NA NA NA NA NA NA ...
 $ SiO4_UDO_Box               : num  2.11 3.85 4.84 4.03 5.14 ...
 $ SiO4_UDO_AtDepth           : num  5.83 6.59 7.92 8.03 6.88 ...
 $ PO4_USF_Box                : num  NA NA NA NA NA NA NA NA NA NA ...
 $ PO4_USF_AtDepth            : num  NA NA NA NA NA NA NA NA NA NA ...
 $ PO4_UDO_Box                : num  0.0576 0.2074 0.3003 0

In [25]:
write.csv(BoxVSAtDepth,"ProcessedDATA/BoxVSatDepth_02.csv")

# CTD DATA

In [26]:
# open a NetCDF file
ctdat <- nc_open("../01Observations/Data/ctd.netcdf")

#print(niskdat)

In [27]:
ctdat_df <- data.frame("Num" = 1:ctdat$dim$unlimited$len)
print('start')
for(i in 1:ctdat$nvars){
    #print(i)
    ctdat_df[paste(attributes(ctdat$var)$names[i])] <- ncvar_get(ctdat, attributes(ctdat$var)$names[i])
}
print('done reading')
head(ctdat_df)

[1] "start"
[1] "done reading"


Num,cruise_no,Cruise_ID1,Cruise_ID2,Year,Month,Day,Date,Latitude,Longitude,...,temp,sal,potemp,sigma_t,sigma_0,O2_ml_L,beam_cp,beam_att,fluor_CTD,fluor_chla
1,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,,36.612,,,,,,,0.173,
2,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.522,36.612,27.522,23.764,23.764,3.964,,,0.176,
3,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.489,36.612,27.488,23.775,23.775,3.978,,,0.166,
4,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.465,36.614,27.464,23.782,23.782,3.984,,,0.164,
5,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.458,36.613,27.456,23.786,23.787,3.990,,,0.170,
6,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.454,36.613,27.452,23.787,23.787,3.992,,,0.175,


In [28]:
cat(names(ctdat_df))

Num cruise_no Cruise_ID1 Cruise_ID2 Year Month Day Date Latitude Longitude press depth temp sal potemp sigma_t sigma_0 O2_ml_L beam_cp beam_att fluor_CTD fluor_chla

In [29]:
ctdat_df$date <- as.Date(ctdat_df$Date)

In [30]:
ctdat_df$press <- as.numeric(ctdat_df$press)
ctdat_df$depth <- as.numeric(ctdat_df$depth)
ctdat_df$temp <- as.numeric(ctdat_df$temp)
ctdat_df$sal <- as.numeric(ctdat_df$sal)
ctdat_df$potemp <- as.numeric(ctdat_df$potemp) 
ctdat_df$sigma_t <- as.numeric(ctdat_df$sigma_t)
ctdat_df$sigma_0 <- as.numeric(ctdat_df$sigma_0)
ctdat_df$O2_ml_L <- as.numeric(ctdat_df$O2_ml_L)
ctdat_df$beam_cp <- as.numeric(ctdat_df$beam_cp)
ctdat_df$beam_att <- as.numeric(ctdat_df$beam_att)
ctdat_df$fluor_CTD <- as.numeric(ctdat_df$fluor_CTD)
ctdat_df$fluor_chla <- as.numeric(ctdat_df$fluor_chla)

In [31]:
ctdat_df[ctdat_df>10000000000]=NA

In [32]:
CTDx258 <- ctdat_df %>%
  group_by(date) %>% # this groups it by the run number 
  #filter(press > 8) %>% # only take values below 8 meters depth to minimze influence of surface anomalies
  mutate(belowmld = sigma_t >= 25.8) %>% # create new column that gives "True" for values with sigma t higher than 25.8
  filter(belowmld == T) %>% # only take "True" values 
  slice(1) %>% # takes the first occurrence
  ungroup() #removes grouping

In [33]:
x258 <- data.frame(date=CTDx258$date, x258depth = CTDx258$depth)

In [34]:
x258$yday <- yday(x258$date)

x258$month <- month(x258$date)

In [35]:
str(x258)

'data.frame':	229 obs. of  4 variables:
 $ date     : Date, format: "1995-11-08" "1995-12-14" ...
 $ x258depth: num  130 44 92 88 38 39 14 53 105 64 ...
 $ yday     : num  312 348 13 45 73 107 130 164 192 219 ...
 $ month    : num  11 12 1 2 3 4 5 6 7 8 ...


In [36]:
write.csv(x258,"ProcessedDATA/x258_02.csv")

# PAR DATA

In [37]:
# read SeaWIFs 8Day extracted file
SeaWIFs <- read.csv('SeaWIFs_PAR_8day_extracted.csv')

In [38]:
date = as.character(paste(SeaWIFs$year,'-',SeaWIFs$month,'-15',sep=''))
SeaWIFs$Date <- as.Date(date, format='%Y-%m-%d')
str(SeaWIFs)

'data.frame':	11540 obs. of  8 variables:
 $ year : int  1997 1997 1997 1997 1997 1997 1997 1997 1997 1997 ...
 $ month: int  9 9 9 9 9 9 9 9 9 9 ...
 $ lon  : num  -64.8 -64.8 -64.7 -64.6 -64.8 ...
 $ lat  : num  10.7 10.7 10.7 10.7 10.6 ...
 $ value: num  31.1 27.3 32.1 38.9 25.7 ...
 $ unit : Factor w/ 1 level "einstein m^-2 day^-1": 1 1 1 1 1 1 1 1 1 1 ...
 $ var  : Factor w/ 1 level "par": 1 1 1 1 1 1 1 1 1 1 ...
 $ Date : Date, format: "1997-09-15" "1997-09-15" ...


In [39]:
PARX <- data.frame(date = SeaWIFs$Date,lat = SeaWIFs$lat,lon = SeaWIFs$lon, value= SeaWIFs$value, unit = SeaWIFs$unit)

In [40]:
PARX$yday <- yday(PARX$date)

PARX$month <- month(PARX$date)

In [41]:
head(PARX)

date,lat,lon,value,unit,yday,month
1997-09-15,10.7326,-64.83918,31.060,einstein m^-2 day^-1,258,9
1997-09-15,10.7326,-64.75515,27.270,einstein m^-2 day^-1,258,9
1997-09-15,10.7326,-64.67112,32.122,einstein m^-2 day^-1,258,9
1997-09-15,10.7326,-64.58709,38.942,einstein m^-2 day^-1,258,9
1997-09-15,10.6482,-64.83918,25.684,einstein m^-2 day^-1,258,9
1997-09-15,10.6482,-64.75515,28.770,einstein m^-2 day^-1,258,9


In [42]:
tail(PARX)

,date,lat,lon,value,unit,yday,month
11535,2010-12-15,10.4794,-64.67112,44.792,einstein m^-2 day^-1,349,12
11536,2010-12-15,10.4794,-64.58709,44.842,einstein m^-2 day^-1,349,12
11537,2010-12-15,10.3950,-64.83918,43.916,einstein m^-2 day^-1,349,12
11538,2010-12-15,10.3950,-64.75515,44.148,einstein m^-2 day^-1,349,12
11539,2010-12-15,10.3950,-64.67112,43.434,einstein m^-2 day^-1,349,12
11540,2010-12-15,10.3950,-64.58709,42.218,einstein m^-2 day^-1,349,12


In [43]:
write.csv(PARX,"ProcessedDATA/PARXSeaWiFS_02.csv")

# EUPHOTIC ZONE

In [44]:
# read MLD depth file
EuZ <- read.csv("par_Light_Level_edit.csv")

# read Date column
EuZ$Date <- as.Date(EuZ$Fecha, format="%m/%d/%y")

# get just month and year for merging later
EuZ$DateMonth <- format(EuZ$Date, format="%Y-%m")

In [45]:
str(EuZ)

'data.frame':	132 obs. of  7 variables:
 $ Campaign : Factor w/ 131 levels "","C118","C119",..: 72 73 74 75 76 77 78 79 80 81 ...
 $ Fecha    : Factor w/ 131 levels "","1/1/98","1/1/99",..: 23 32 2 42 54 67 89 101 111 121 ...
 $ X0.10.   : num  59 59 46.5 NA 25.5 74 71 60 71.5 74 ...
 $ X1.      : num  48.5 34 33 NA 14.5 45 45 41.5 48 57.5 ...
 $ X10.     : num  23 13.5 14 21 6.5 21 23.5 17.5 29 27 ...
 $ Date     : Date, format: "1997-11-01" "1997-12-01" ...
 $ DateMonth: chr  "1997-11" "1997-12" "1998-01" "1998-02" ...


In [46]:
tail(EuZ)

,Campaign,Fecha,X0.10.,X1.,X10.,Date,DateMonth
127,C185,10/4/11,NA,58.6,28.7,2011-10-04,2011-10
128,C186,11/18/11,81.0,57.0,36.3,2011-11-18,2011-11
129,C187,1/10/12,NA,29.7,17.1,2012-01-10,2012-01
130,C188,2/7/12,NA,40.2,19.2,2012-02-07,2012-02
131,C189,3/16/12,23.1,12.4,4.6,2012-03-16,2012-03
132,,,NA,49.2,5.3,NA,NA


In [47]:
EuZX <- data.frame(date=EuZ$Date, X0.1=EuZ$X0.10.)

In [48]:
EuZX$yday <- yday(EuZX$date)

EuZX$month <- month(EuZX$date)

In [49]:
str(EuZX)

'data.frame':	132 obs. of  4 variables:
 $ date : Date, format: "1997-11-01" "1997-12-01" ...
 $ X0.1 : num  59 59 46.5 NA 25.5 74 71 60 71.5 74 ...
 $ yday : num  305 335 1 32 60 91 152 182 213 244 ...
 $ month: num  11 12 1 2 3 4 6 7 8 9 ...


In [50]:
write.csv(EuZX,"ProcessedDATA/EuZXEuphoticZoneDepth_02.csv")